Implementation of [Rodriguez](https://ieeexplore.ieee.org/document/5475260) (sec. III)

In [30]:
# imports & constants
import numpy as np
import matplotlib.pyplot as plt
from math import floor, ceil

SEED = 0
np.random.seed(SEED)

In [31]:
# generate single tone
amplitude = 1
f_tone = 60.98
phi_0 = 0
f_s = 1_200
M = 200
N_DFT = 20_000 + 1

n = np.arange(M)
s_tone = amplitude*np.cos(2 * np.pi * f_tone / f_s * n + phi_0)
del n

In [32]:
# A. Phase Estimation Using the DFT
hann_window = np.hanning(M)
x = s_tone * hann_window
X = np.fft.fft(x, n=N_DFT)
# freqs = np.fft.fftfreq(NDFT, 1/f_s)
# f_DFT = freqs[k_peak]

k_peak = np.argmax(np.abs(X[:N_DFT//2]))
f_DFT = k_peak*f_s/N_DFT

phi_DFT = np.angle(X[k_peak]) # % (2*np.pi)

In [33]:
# B. The Novel Phase Estimation Method
# 1
s_dash = f_s * np.diff(s_tone)
s_tone = s_tone[1:]

# 2
hann_window = np.hanning(M-1)
x = s_tone * hann_window
x_dash = s_dash * hann_window

# 3
X = np.fft.fft(x, n=N_DFT)
X_dash = np.fft.fft(x_dash, n=N_DFT)

# 4
abs_X = np.abs(X)
abs_X_dash = np.abs(X_dash)
k_peak = np.argmax(abs_X)

# 5
F_k_peak = (np.pi * k_peak) / (N_DFT * np.sin(np.pi * k_peak / N_DFT))

DFT_0_peak = abs_X[k_peak]
DFT_1_peak = F_k_peak * abs_X_dash[k_peak]

# 6
f_DFT1 = 1 / (2 * np.pi) * DFT_1_peak / DFT_0_peak
k_DFT1 = N_DFT * f_DFT1 / f_s

# validating
if not -.5 <= k_DFT1 - k_peak < .5:
    raise ValueError(f"invalid result: k_peak: {k_peak}, k_DFT1: {k_DFT1}")

In [34]:
# continuing with the method

omega_0 = 2*np.pi*f_DFT1/f_s
k_low = floor(k_DFT1)
k_high = ceil(k_DFT1)
theta_low = np.angle(X_dash[k_low])
theta_high = np.angle(X_dash[k_high])

theta = (k_DFT1 - k_low) * (theta_high - theta_low) / (k_high - k_low) + theta_low
# theta = np.interp(k_DFT1, [k_low, k_high], [theta_low, theta_high])

numerator = np.tan(theta) * (1 - np.cos(omega_0)) + np.sin(omega_0)
denominator = 1 - np.cos(omega_0) - np.tan(theta) * np.sin(omega_0)
phi_DFT1 = np.arctan(numerator / denominator)
# phi_DFT1 = np.arctan2(numerator, denominator)

# choosing phi_DFT1 closer to phi_DFT
_sym_intvl = lambda x: (x + np.pi) % (2*np.pi) - np.pi
_phi_DFT1 = _sym_intvl(phi_DFT1 + np.pi)
if np.abs(_phi_DFT1 - phi_DFT) < np.abs(phi_DFT1- phi_DFT):
    phi_DFT1 = _phi_DFT1


In [35]:
pattern = " .6f"
print(f"f_DFT:    {f_DFT:{pattern}}")
print(f"f_DFT1:   {f_DFT1:{pattern}}")
print(f"f_tone:   {f_tone:{pattern}}")
print()
print(f"phi_DFT:  {np.degrees(phi_DFT):{pattern}} degrees")
print(f"phi_DFT1: {np.degrees(phi_DFT1):{pattern}} degrees")
print(f"phi_0:    {np.degrees(phi_0):{pattern}} degrees")
print()
print(f"e_f_DFT:    {np.abs(f_DFT-f_tone)/f_tone*100:.4f} %")
print(f"e_f_DFT1:   {np.abs(f_DFT1-f_tone)/f_tone*100:.4f} %")
print(f"e_phi_DFT:  {np.abs(phi_DFT-phi_0):.4f} degrees")
print(f"e_phi_DFT1: {np.abs(phi_DFT1-phi_0):.4f} degrees")



f_DFT:     60.956952
f_DFT1:    60.980924
f_tone:    60.980000

phi_DFT:   0.688897 degrees
phi_DFT1: -0.028129 degrees
phi_0:     0.000000 degrees

e_f_DFT:    0.0378 %
e_f_DFT1:   0.0015 %
e_phi_DFT:  0.0120 degrees
e_phi_DFT1: 0.0005 degrees
